# Setup stuff: Connectivity

In [ ]:

#  This file also includes;
#     .  tabulate
#         Eg.,   print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Setting display options

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Setup stuff: Debugging 

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


#  Actual analytics ..

In [ ]:

from katana import remote
from katana.remote import analytics


l_cntr = 20

print("--")


# Iteration/test 1 ..

In [ ]:

#  Running analytics.sssp() .. iteration 1 ..
#
#  This used to work, and it works no longer. See comments after this cell.
#

l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)
   #
l_weight_property     = "DISTANCE"                 #  Edge property

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
      #
   source               = l_source_nodeid,
      #
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


In [ ]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


#  Sample output,
#
#     +----+-----------------+-------------+-----------+------------------+--------+----------+
#     |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
#     |----+-----------------+-------------+-----------+------------------+--------+----------|
#     |  0 |               0 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
#     |  1 |               1 | ['Airport'] | Airport   | Denver           | DEN    | node     |
#     |  2 | 281474976710656 | ['Airport'] | Airport   | San Jose         | SJC    | node     |
#     |  3 | 562949953421312 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    | node     |
#     +----+-----------------+-------------+-----------+------------------+--------+----------+
#     
#     +----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
#     |    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
#     |----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
#     |  0 | 281474976710656 | ['Airport']    |               0 | FLIES_TO  |          948 |            1 | FLIES_TO |               1 | ['Airport']      | edge     |
#     |  1 | 562949953421312 | ['Airport']    |               2 | FLIES_TO  |          886 |            1 | FLIES_TO |               1 | ['Airport']      | edge     |
#     |  2 | 562949953421312 | ['Airport']    |               1 | FLIES_TO  |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
#     |  3 | 562949953421312 | ['Airport']    | 281474976710656 | FLIES_TO  |         1829 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
#     |  4 |               1 | ['Airport']    | 281474976710657 | FLIES_TO  |          948 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
#     |  5 |               1 | ['Airport']    | 562949953421312 | FLIES_TO  |          886 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     |
#     |  6 |               0 | ['Airport']    | 562949953421313 | FLIES_TO  |           66 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     |
#     +----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+


#  Results,
#
#     .  This block used to work.
#        The "id" for nodes was supplied as a unique character string, and our DataFrame importer would
#        overwrite that value with an integer.
#        Apparently that behavior has changed.
#        Okay. Good
#
#     .  What we received is that the source (source node id) was not found, so no result_property_name
#        was output/created.
#
#         Should an error have been output instead ??



# Iteration/test 2 ..

In [ ]:

#  Running analytics.sssp() .. iteration 2 ..
#
#  Change source (source node id) to "SJC"
#

l_source_nodeid       = "SJC"                      #  id of node to start on (this is SJC)
   #
l_weight_property     = "DISTANCE"                 #  Edge property

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
      #
   source               = l_source_nodeid,
      #
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))

#  Error received,
#
#     InvalidRequestError: b'{"code":400,"message":"json: invalid uint64 value; should be encoded as a string"}'
